In [6]:
using DelimitedFiles
using Statistics

In [14]:
irisPath = "/home/minhhn/Documents/MachineLearning/Dataset/Iris/iris-train.txt"

# read data function
function readData(path)
    A = readdlm(path)
    y = Int.(A[:,1])
    X = A[:,2:end]
    (X, y)
end

X,y = readData(irisPath)

([5.1 3.5 1.4 0.2; 4.7 3.2 1.3 0.2; … ; 6.2 3.4 5.4 2.3; 5.9 3.0 5.1 1.8], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [15]:
function train(X, y)
    K = length(unique(y))
    N, D = size(X)
    μ = zeros(D, K)
    σ = zeros(D, K)
    θ = zeros(K) # prior, θ[k] = P(y=k)
    for k=1:K
        idk = (y .== k)
        Xk = X[idk, :]
        μ[:, k] = mean(Xk, dims = 1)
        σ[:, k] = std(Xk, dims = 1)
        θ[k] = sum(idk)/N
    end
    (μ, σ, θ) # \mu, \sigma, \theta 
end

μ, σ, θ = train(X, y)

([5.013953488372093 5.902272727272727 6.560465116279069; 3.40232558139535 2.759090909090909 2.9697674418604647; 1.4627906976744185 4.229545454545454 5.532558139534883; 0.24186046511627904 1.3136363636363635 2.016279069767442], [0.35495230756018964 0.5205045121282594 0.6388753268658622; 0.39004784327604836 0.3112750834709582 0.33419048155557596; 0.15278876768419214 0.48348518914067395 0.5664092959926881; 0.10961525384712201 0.20413277609154645 0.28278397657688786], [0.33076923076923076, 0.3384615384615385, 0.33076923076923076])

$\log P(y = k|x) = \sum_{j=1}^D \log P(x_j|y=k) + \log P(y=k)$

$ = \sum_{j=1}^D \left[ -\log(\sqrt{2\pi}\sigma) - \frac{1}{2\sigma_{jk}^2}(x_j - \mu_{jk})^2 \right] + \log \theta_k $

$ \propto -\sum_{j=1}^D \left[\log (\sigma_{jk}) + \frac{1}{2\sigma_{jk}^2}(x_j - \mu_{jk})^2 \right] + \log \theta_k $

In [20]:
function classify(μ, σ, θ, x)
    K = length(θ)
    p = zeros(K) # log posterior distribution
    for k=1:K
        p[k] = -sum(log.(σ[:,k]) + (x - μ[:,k]) .^2 ./ (2*σ[:,k].^2)) + log(θ[k])
    end
    argmax(p)
end


classify (generic function with 1 method)

In [22]:
classify(μ, σ, θ, X[end,:])

3

In [ ]:
z = map(i -> classify(μ, σ, θ, X[i,:]), 1:length(y))

In [24]:
ŷ = [classify(μ, σ, θ, X[i,:]) for i=1:length(y)] # y\hat
training_accuracy = sum(ŷ .== y)/length(y)

0.9538461538461539